<a href="https://colab.research.google.com/github/marcelo-vitti/coffee_and_currency_rates_case/blob/main/notebooks/Pismo_Analytics_Engineer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get Currency Rates to SQLite Table

In [1]:
import pandas as pd
import requests
import json
import datetime

from datetime import date, timedelta

In [2]:
API_KEY = "58b5b643ff774a55babfbc8c21d66914"
API_LINK = "https://openexchangerates.org/api/historical/"
START_DATE = date(2021, 1, 1)
END_DATE = date(2023, 3, 29)
CURRENCY_SOURCE = "USD"
DESIRED_CURRENCIES = ["BRL", "CLP", "EUR"]

In [3]:
def enable_log_messages():
        
  import logging

  logger = logging.getLogger()
  logger.setLevel(logging.INFO)

  return logging

In [4]:
def daterange(start_date, end_date):

    for n in range(int((end_date - start_date).days)):
      
        yield start_date + timedelta(n)

In [5]:
data = []

for single_date in daterange(START_DATE, END_DATE):

  api_call_date = single_date.strftime("%Y-%m-%d")

  logging = enable_log_messages()
  logging.info(f"[INFO] Getting currency rate for date {api_call_date}.")

  url = f"{API_LINK}{api_call_date}.json?app_id={API_KEY}&show_alternative=false&prettyprint=false"

  payload = {}
  headers= {
    "apikey": API_KEY
  }

  response = requests.request("GET", url, headers=headers, data = payload)

  response_json = response.json()

  # On this for loop I am creating the table in the format that the challenge suggests "Esses dados devem ser 
  # estruturados (Moeda, Câmbio e Data)", but, maybe a better approach to make it easier to query would be 
  # to have just one line per date with all currencies instead of one line per date per currency.
  for key, value in response_json["rates"].items():

    if key in DESIRED_CURRENCIES:

      return_dict = {
        "activity_date": api_call_date,
        "currency": key,
        "rate": value,
      } 

      data.append(return_dict)


INFO:root:[INFO] Getting currency rate for date 2021-01-01.
INFO:root:[INFO] Getting currency rate for date 2021-01-02.
INFO:root:[INFO] Getting currency rate for date 2021-01-03.
INFO:root:[INFO] Getting currency rate for date 2021-01-04.
INFO:root:[INFO] Getting currency rate for date 2021-01-05.
INFO:root:[INFO] Getting currency rate for date 2021-01-06.
INFO:root:[INFO] Getting currency rate for date 2021-01-07.
INFO:root:[INFO] Getting currency rate for date 2021-01-08.
INFO:root:[INFO] Getting currency rate for date 2021-01-09.
INFO:root:[INFO] Getting currency rate for date 2021-01-10.
INFO:root:[INFO] Getting currency rate for date 2021-01-11.
INFO:root:[INFO] Getting currency rate for date 2021-01-12.
INFO:root:[INFO] Getting currency rate for date 2021-01-13.
INFO:root:[INFO] Getting currency rate for date 2021-01-14.
INFO:root:[INFO] Getting currency rate for date 2021-01-15.
INFO:root:[INFO] Getting currency rate for date 2021-01-16.
INFO:root:[INFO] Getting currency rate f

In [6]:
df_currency_rates = pd.DataFrame(data)
df_currency_rates

,activity_date,currency,rate
0,2021-01-01,BRL,5.193400
1,2021-01-01,CLP,710.499390
2,2021-01-01,EUR,0.822681
3,2021-01-02,BRL,5.193400
4,2021-01-02,CLP,710.499390
...,...,...,...
2446,2023-03-27,CLP,806.230000
2447,2023-03-27,EUR,0.925365
2448,2023-03-28,BRL,5.165900
2449,2023-03-28,CLP,796.825397


In [7]:
df_currency_rates["activity_date"] = df_currency_rates["activity_date"].astype('datetime64[ns]')

In [8]:
df_currency_rates["is_month_end"] = df_currency_rates.activity_date.apply(lambda x: True if x.is_month_end else False)
df_currency_rates["is_year_end"] = df_currency_rates.activity_date.apply(lambda x: True if x.is_year_end else False)

In [9]:
import sqlite3

conn = sqlite3.connect("coffee_and_currencies.db")

In [10]:
df_currency_rates.to_sql("currency_rates_data", conn, if_exists="replace")

2451

In [11]:
conn.execute(
    """
    create table IF NOT EXISTS currency_rates as 
    select * from currency_rates_data
    """)

In [12]:
conn.commit()


In [13]:
conn.close()

# Get Coffee to SQLite Table

In [14]:
coffee = pd.read_csv("https://raw.githubusercontent.com/marcelo-vitti/coffee_and_currency_rates_case/main/external_sources/coffee.csv")

In [15]:
conn = sqlite3.connect("coffee_and_currencies.db")

coffee.to_sql("coffee_data", conn, if_exists="replace")

conn.execute(
    """
    create table IF NOT EXISTS coffee as 
    select * from coffee_data
    """)

conn.commit()


In [16]:
conn.close()

# Get SQL Queries results

In [129]:
# I had a problem getting the query formatting when I was getting it from the Github so I put it hardcoded here.

coffee_per_year_and_rates_sql = """
WITH coffee_and_currency_base AS (

  SELECT
    strftime("%Y", coffee.Date) AS activity_year,
    coffee.Date,
    coffee.Volume,  
    AVG(
    CASE
      WHEN currency_rates.currency = "USDBRL" THEN currency_rates.rate
      ELSE NULL
    END) AS brl_usd_rate,
    AVG(
      CASE
        WHEN currency_rates.currency = "USDCLP" THEN currency_rates.rate
        ELSE NULL
      END) AS clp_usd_rate,
    AVG(
      CASE
        WHEN currency_rates.currency = "USDEUR" THEN currency_rates.rate
        ELSE NULL
      END) AS eur_usd_rate,
    ROW_NUMBER() OVER (
      PARTITION BY strftime("%Y", coffee.Date) 
      ORDER BY currency_rates.activity_date DESC
    ) AS row_number_desc

  FROM coffee

    LEFT JOIN currency_rates ON coffee.Date = DATE(currency_rates.activity_date)

  GROUP BY activity_year, Date, Close

)

SELECT 
  activity_year,
  SUM(CASE WHEN row_number_desc = 1 THEN brl_usd_rate ELSE 0 END) AS brl_usd_rate,
  SUM(CASE WHEN row_number_desc = 1 THEN clp_usd_rate ELSE 0 END) AS clp_usd_rate,
  SUM(CASE WHEN row_number_desc = 1 THEN eur_usd_rate ELSE 0 END) AS eur_usd_rate,
  SUM(Volume)                                                     AS total_negotiated_coffee

FROM coffee_and_currency_base

GROUP BY activity_year
"""


highest_coffee_volume_and_currency_closing_sql = """
-- If we have a currency_rates table with all currencies in just one line we wouldn't need to do this treatment, it would be easier.
-- I am doing an inner join to bring only data from coffee in dates that we have currency data.

SELECT
  coffee.Date AS activity_date,
  coffee.High AS higher_coffee_volume,
  AVG(
    CASE
      WHEN currency_rates.currency = "BRL" THEN currency_rates.rate
      ELSE NULL
    END) AS brl_usd_rate,
  AVG(
    CASE
      WHEN currency_rates.currency = "CLP" THEN currency_rates.rate
      ELSE NULL
    END) AS clp_usd_rate,
  AVG(
    CASE
      WHEN currency_rates.currency = "EUR" THEN currency_rates.rate
      ELSE NULL
    END) AS eur_usd_rate
  
FROM coffee

  LEFT JOIN currency_rates ON coffee.Date = DATE(currency_rates.activity_date)

GROUP BY coffee.Date, coffee.High
"""


negotiated_coffee_average_sql = """
SELECT
  "yearly"                    AS analysis_interval,
  strftime("%Y", coffee.Date) AS activity_date,
  AVG(Volume)                 AS average_of_negotiated_coffee

FROM coffee

GROUP BY strftime("%Y", coffee.Date)

UNION ALL

SELECT
  "monthly"                      AS analysis_interval,
  strftime("%Y-%m", coffee.Date) AS activity_date,
  AVG(Volume)                    AS average_of_negotiated_coffee

FROM coffee

GROUP BY strftime("%Y-%m", coffee.Date)
"""


In [130]:
conn = sqlite3.connect("coffee_and_currencies.db")

**coffee_per_year_and_rates:** Maior volume negociado de café no dia e as cotações de fechamento

In [131]:
df_coffee_per_year_and_rates = pd.read_sql_query(coffee_per_year_and_rates_sql, conn)
df_coffee_per_year_and_rates.head()

,activity_year,brl_usd_rate,clp_usd_rate,eur_usd_rate,total_negotiated_coffee
0,2019,0,0,0,4011645
1,2020,0,0,0,3078378
2,2021,0,0,0,3127536
3,2022,0,0,0,2374254


**highest_coffee_volume_and_currency_closing:** Total de café negociado por ano e as cotações

In [132]:
df_highest_coffee_volume_and_currency_closing = pd.read_sql_query(highest_coffee_volume_and_currency_closing_sql, conn)
df_highest_coffee_volume_and_currency_closing.head()

,activity_date,higher_coffee_volume,brl_usd_rate,clp_usd_rate,eur_usd_rate
0,2019-01-02,102.65,NaN,NaN,NaN
1,2019-01-03,103.25,NaN,NaN,NaN
2,2019-01-04,103.00,NaN,NaN,NaN
3,2019-01-07,103.45,NaN,NaN,NaN
4,2019-01-08,106.05,NaN,NaN,NaN


**negotiated_coffee_average:** Média de volume negociado mensal e anual 

In [133]:
df_negotiated_coffee_average = pd.read_sql_query(negotiated_coffee_average_sql, conn)
df_negotiated_coffee_average.head()

,analysis_interval,activity_date,average_of_negotiated_coffee
0,yearly,2019,15919.226190
1,yearly,2020,12167.501976
2,yearly,2021,12410.857143
3,yearly,2022,14217.089820
4,monthly,2019-01,21827.476190


In [134]:
conn.close()

# Save SQL queries results in Google Drive

In [135]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [136]:
base_path = "/content/drive/MyDrive/PismoCase/csv"

In [137]:
df_coffee_per_year_and_rates.to_csv(f"{base_path}/coffee_per_year_and_rates.csv", index=False)
df_highest_coffee_volume_and_currency_closing.to_csv(f"{base_path}/highest_coffee_volume_and_currency_closing.csv", index=False)
df_negotiated_coffee_average.to_csv(f"{base_path}/negotiated_coffee_average.csv", index=False)
df_currency_rates.to_csv(f"{base_path}/currency_rates.csv", index=False)